In [37]:
import numpy as np
import os 
import matplotlib.pyplot as plt
import librosa 
import joblib

Ler os arquivos de áudio para coletar os coeficientes e os os labels. Os labels serão divididos da seguinte maneira:
- Anger: 0
- Disgust: 1
- Fear: 2
- Happy: 3
- Neutral: 4
- Sad: 5

In [46]:
base_dir = 'C:/Users/yfrom/OneDrive/Área de Trabalho/UFRJ/10/Proc voz/Filtro-de-ofenças/Dados'

In [43]:
data, sampling_rate = librosa.load('C:/Users/yfrom/OneDrive/Área de Trabalho/UFRJ/10/Proc voz/Dados/anger.wav')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (542650039.py, line 1)

Vou ler apenas 150 audios de cada emoção pq eu não quero queimar meu computador

In [47]:
label_map = {'anger': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5}
lst = []
number_of_audios_per_emotion = 1000
for emotion in label_map.keys():
    folder_path = os.path.join(base_dir, emotion)
    audios_number = 0
    # Iterate over the audio files in the folder
    for audio_file in os.listdir(folder_path):
        # print(os.path.join(folder_path, audio_file))
        if audios_number <= number_of_audios_per_emotion:
            X, sample_rate = librosa.load(os.path.join(folder_path, audio_file))
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)         
            file = label_map[emotion]
            arr = mfccs, file
            lst.append(arr)        
            audios_number += 1
        else: break
    print(audios_number, emotion, 'audio files loaded')

1001 anger audio files loaded
1001 disgust audio files loaded
1001 fear audio files loaded
1001 happy audio files loaded
1001 neutral audio files loaded
1001 sad audio files loaded


In [48]:
X, y = zip(*lst)

In [49]:
X = np.asarray(X)
y = np.asarray(y)


X.shape, y.shape

((6006, 40), (6006,))

In [53]:
import joblib

X_name = 'X.joblib'
y_name = 'y.joblib'
save_dir = 'C:/Users/yfrom/OneDrive/Área de Trabalho/UFRJ/10/Proc voz/Filtro-de-ofenças/Insult-filter/data'

savedX = joblib.dump(X, os.path.join(save_dir, X_name))
savedy = joblib.dump(y, os.path.join(save_dir, y_name))

In [54]:
X = joblib.load('data/X.joblib')
y = joblib.load('data/y.joblib')

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Neural Network

In [56]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [57]:
x_traincnn.shape, x_testcnn.shape

((4024, 40, 1), (1982, 40, 1))

In [102]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
model = Sequential()

model.add(Conv1D(128, 5,padding='same',  input_shape=(40,1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(6)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(6))
model.add(Activation('softmax'))
opt = optimizers.RMSprop(learning_rate=0.00005, rho=0.9, epsilon=None, decay=0.0)

c:\Users\yfrom\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\yfrom\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [103]:
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_42 (Conv1D)              │ (None, 40, 128)        │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 40, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 40, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_37 (MaxPooling1D) │ (None, 6, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_43 (Conv1D)              │ (None, 6, 128)         │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 6, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 6, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 6)              │         4,614 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 6)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 87,430 (341.52 KB)

 Trainable params: 87,430 (341.52 KB)

 Non-trainable params: 0 (0.00 B)

In [104]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [19]:
print("x_traincnn shape:", x_traincnn.shape)
print("y_train shape:", y_train.shape)
print("x_testcnn shape:", x_testcnn.shape)
print("y_test shape:", y_test.shape)

x_traincnn shape: (607, 40, 1)
y_train shape: (607,)
x_testcnn shape: (299, 40, 1)
y_test shape: (299,)


In [106]:
cnnhistory = model.fit(x_traincnn, y_train, batch_size=40, epochs=10, validation_data=(x_testcnn, y_test))

Epoch 1/10


AttributeError: 'NoneType' object has no attribute 'items'

## Modelo 4

Acuracia de 0.60

In [58]:
        
        callback = keras.callbacks.EarlyStopping(monitor='loss',patience= 3)
        model = Sequential()
        model.add(Conv1D(64, 5, padding='same',
                         input_shape=(40, 1)))
        model.add(Activation('relu'))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(6))
        model.add(Activation('softmax'))

        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['accuracy'])

        cnn_history = model.fit(x_traincnn, y_train,
                               batch_size=16, epochs=1000,
                               validation_data=(x_testcnn, y_test), callbacks=[callback])

Epoch 1/1000


c:\Users\yfrom\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


252/252 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2412 - loss: 6.7679 - val_accuracy: 0.3305 - val_loss: 3.7169
Epoch 2/1000
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3831 - loss: 2.4516 - val_accuracy: 0.3804 - val_loss: 2.0564
Epoch 3/1000
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4453 - loss: 1.4775 - val_accuracy: 0.5283 - val_loss: 1.2302
Epoch 4/1000
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5508 - loss: 1.2350 - val_accuracy: 0.5187 - val_loss: 1.2402
Epoch 5/1000
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5365 - loss: 1.2210 - val_accuracy: 0.5293 - val_loss: 1.2631
Epoch 6/1000
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5725 - loss: 1.1787 - val_accuracy: 0.5267 - val_loss: 1.2115
Epoch 7/1000
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5710 - loss: 1.1539 - val_accuracy: 0.5742 - val_loss: 1.1388
Epoch 8/1000
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5759 - loss: 1.1266 - val_accuracy:

In [59]:
predictions = np.argmax(model.predict(x_testcnn),axis=1)

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [60]:
from sklearn.metrics import classification_report
report = classification_report(y_test, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.49      0.52      0.50       358
           1       0.62      0.55      0.58       340
           2       0.81      0.69      0.75       333
           3       0.47      0.86      0.61       317
           4       0.79      0.56      0.65       316
           5       0.65      0.45      0.53       318

    accuracy                           0.60      1982
   macro avg       0.64      0.60      0.60      1982
weighted avg       0.64      0.60      0.60      1982



## Modelo https://www.kaggle.com/code/shivamburnwal/speech-emotion-recognition?scriptVersionId=34958802&cellId=52

Acurácia de 0.79

In [72]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

In [112]:
model=Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x_traincnn.shape[1], 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.2))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(units=6, activation='softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model.summary()

c:\Users\yfrom\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_48 (Conv1D)              │ (None, 40, 256)        │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 40, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_42 (MaxPooling1D) │ (None, 20, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_49 (Conv1D)              │ (None, 20, 256)        │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_43 (MaxPooling1D) │ (None, 10, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_50 (Conv1D)              │ (None, 10, 128)        │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_44 (MaxPooling1D) │ (None, 5, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 5, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_51 (Conv1D)              │ (None, 5, 64)          │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_45 (MaxPooling1D) │ (None, 3, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_16 (Flatten)            │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 32)             │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 541,990 (2.07 MB)

 Trainable params: 541,414 (2.07 MB)

 Non-trainable params: 576 (2.25 KB)

In [113]:
from tensorflow.keras.utils import to_categorical

In [114]:
y_train_encoded = to_categorical(y_train, num_classes=6)
y_test_encoded = to_categorical(y_test, num_classes=6)

In [115]:
y_train_encoded.shape

(4024, 6)

In [118]:
callback = keras.callbacks.EarlyStopping(monitor='loss',patience= 3)
history=model.fit(x_traincnn, y_train_encoded, batch_size=60, epochs=100, validation_data=(x_testcnn, y_test_encoded), callbacks=[callback])

Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.6314 - loss: 0.9845 - val_accuracy: 0.6418 - val_loss: 0.9439
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.6654 - loss: 0.8832 - val_accuracy: 0.6448 - val_loss: 0.9241
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.7112 - loss: 0.7904 - val_accuracy: 0.6645 - val_loss: 0.9222
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.6795 - loss: 0.8425 - val_accuracy: 0.6796 - val_loss: 0.8556
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7099 - loss: 0.7817 - val_accuracy: 0.7003 - val_loss: 0.8344
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7353 - loss: 0.7219 - val_accuracy: 0.6519 - val_loss: 0.9191
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.7559 - loss: 0.6805 - val_accuracy: 0.7089 - val_loss: 0.7753
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.7535 - loss: 0.6842 - val_accuracy: 0.

In [119]:
predictions = np.argmax(model.predict(x_testcnn),axis=1)

62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [120]:
predictions_encoded = to_categorical(predictions, num_classes=6)

In [122]:
from sklearn.metrics import classification_report
report = classification_report(y_test_encoded, predictions_encoded)
print(report)

              precision    recall  f1-score   support

           0       0.67      0.74      0.70       358
           1       0.75      0.73      0.74       340
           2       0.91      0.83      0.87       333
           3       0.78      0.77      0.77       317
           4       0.85      0.78      0.81       316
           5       0.66      0.73      0.69       318

   micro avg       0.76      0.76      0.76      1982
   macro avg       0.77      0.76      0.77      1982
weighted avg       0.77      0.76      0.76      1982
 samples avg       0.76      0.76      0.76      1982



In [125]:
MODEL_DIR_PATH = 'models'

In [126]:
        if not os.path.isdir(MODEL_DIR_PATH):
            os.makedirs(MODEL_DIR_PATH)
        model_path = os.path.join(MODEL_DIR_PATH, 'Emotion_Voice_Detection_4_layers_cnn.keras')
        model.save(model_path)
        print('Saved trained model at %s ' % model_path)

Saved trained model at models\Emotion_Voice_Detection_4_layers_cnn.keras 
